The ultimate goal is to perform holoVQE. \
It is divided into two parts: first part include functions to calculate expectation value of H; the second part include functions for the optimization\
More specifically, part 1 includes: functions to construct MPS from the unitary matrices, functions to construct H as MPO and functions to calculate the expectation value of H.\
Part 1:

In [417]:
def Uni_to_ten(uni_matrix, site_num, d_bond, d_phys_index, L):
    '''
    construct the matrix corresponding to physics leg index in local Hilbert space for translational invariant 1d chain
    
    Parameters
    ----------
    uni_matrix: numpy.ndarray
        the unitary matrix already constructed
    site_num: int
        the position of the site
    d_bond: int
        bond dimension
    d_phys_index: int
        the index in the physical dimension
    L: int
        the total number of sites
    '''
    if site_num == 0:
        res_matrix = uni_matrix[0:1, d_phys_index * d_bond: (d_phys_index+1)*d_bond]
    elif site_num == L-1:
        res_matrix = uni_matrix[0:d_bond, d_phys_index * d_bond: d_phys_index * d_bond + 1]
    else:
        res_matrix = uni_matrix[0:d_bond, d_phys_index * d_bond: (d_phys_index + 1) * d_bond]
    return res_matrix

In [418]:
import tenpy.linalg.np_conserved as npc
def Site_Ten(uni_matrix, site_num, d_bond, d_phys, L):
    '''
    construct the 3-leg tensor for one site of the translational invariant 1d chain
    
    Parameters
    ----------
    d_phys: dimension of local Hilbert space
    
    returns the npc.Array type, which contains the tensor and its leg labels
    '''
    res_tensor = npc.Array.from_ndarray_trivial([Uni_to_ten(uni_matrix, site_num, d_bond, i, L) \
                                                 for i in range(0,d_phys)],labels=['p'+str(site_num),'vL','vR'])
    return res_tensor

In [420]:
def MPO_H_contract(H_mpo, psi, L):
    '''
    calculate the expectation value of H by doing the contraction over the full MPS

    Parameters
    ----------
    H_mpo: MPO type
    psi: MPS type
    L: integer
        the total number of sites
    '''
    from tenpy.networks.mpo import MPO, MPOEnvironment
    env = MPOEnvironment(psi, H_mpo, psi)
    E = env.full_contraction(L-1)
    return E

tests:

In [421]:
import tenpy
import numpy as np
import tenpy.linalg.np_conserved as npc
from scipy.stats import unitary_group

# for purpose of test, generate arbitrary unitary matrices
x1 = unitary_group.rvs(4)
x2 = unitary_group.rvs(4)
x3 = unitary_group.rvs(4)
# test Uni_to_ten
x1_1 = Uni_to_ten(x,0,2,1,2)

# test Site_ten
x1_t = Site_Ten(x1,0,2,2,2)
x2_t = Site_Ten(x2,1,2,2,2)
x1_a = npc.Array.to_ndarray(x1_t)
x2_a = npc.Array.to_ndarray(x2_t)

print(x1)
print('******')
print(x2)
print('******')
print(x1_1)
print('******')
print(x1_t.rank)
print(x1_a)
print('******')
print(x2_a)

[[-0.23125719+0.04129987j -0.29393868-0.66907925j  0.02176497-0.27134754j
   0.12285892+0.56705373j]
 [ 0.45381778+0.16958233j -0.36314481-0.24699069j  0.46768753+0.57244006j
  -0.16040729-0.0162159j ]
 [-0.71793713+0.15038421j -0.35504616+0.06707238j  0.22634373+0.01592496j
  -0.42273869-0.31812151j]
 [ 0.16634691-0.37998059j -0.36392502+0.1002824j  -0.56330378+0.10335043j
  -0.55711215+0.2169771j ]]
******
[[-0.03860329-0.09608574j  0.07412494+0.57144332j  0.35211546+0.27327506j
   0.08469991+0.67186063j]
 [-0.27726637-0.01897198j  0.48597737+0.06160767j -0.51439668+0.63023451j
   0.07911564-0.12138842j]
 [ 0.11443546+0.32189095j -0.4903475 +0.3544319j   0.01568398+0.27434115j
   0.54314667-0.38302838j]
 [-0.81727631-0.35691864j -0.21825525+0.11954293j  0.24643835-0.05740254j
  -0.0623579 -0.27354566j]]
******
[[ 0.02632271-0.18775696j -0.24329483+0.5502272j ]]
******
3
[[[-0.23125719+0.04129987j -0.29393868-0.66907925j]]

 [[ 0.02176497-0.27134754j  0.12285892+0.56705373j]]]
******


In [422]:
# test for MPS.from_full
from tenpy.networks.site import BosonSite
from tenpy.models.lattice import Chain
import tenpy.linalg.np_conserved as npc
site = BosonSite(conserve = None)
L = 2
lat = Chain(L, site, bc_MPS = 'finite')
state = npc.tensordot(x_t, x_t1, ('vR','vL'))
psi = MPS.from_full(lat.mps_sites(), state)

print(site)
print(lat)
print(type(state))
print(type(psi))
print(psi)

BosonSite(1, None, 0.000000)
<class 'tenpy.linalg.np_conserved.Array'>
<class 'tenpy.networks.mps.MPS'>
MPS, L=2, bc='finite'.
chi: [2]
sites: BosonSite(1, None, 0.000000) BosonSite(1, None, 0.000000)
forms: (1.0, 0.0) (0.0, 1.0)


In [479]:
from tenpy.networks.mps import MPS
from tenpy.networks.site import BosonSite, SpinHalfSite
from tenpy.models.lattice import Chain
import tenpy.linalg.np_conserved as npc

def MPS_state(UM_list, d_phys, d_bond, L):
    '''
    construct the Matrix Product state for the whole chain (1d translational invariant bosonic chain) (can be adjusted)
    
    Parameters
    ----------
    UM_list: list of numpy.ndarray matrices
    
    returns the MPS type in TenPy
    '''
    site = BosonSite(Nmax = 1, conserve = None)
    # site = SpinHalfSite(conserve = None)
    lat = Chain(L, site, bc_MPS = 'finite')
    state = Site_Ten(UM_list[0], 0, d_phys, d_bond, L)
    for i in range(1, L):
        state1 = Site_Ten(UM_list[i], i, d_phys, d_bond, L)
        state = npc.tensordot(state, state1, ('vR','vL'))
    psi = MPS.from_full(lat.mps_sites(), state, 'B')
    # other methods to consider, MPS.from_Bflat, MPS
    # problem with MPS.from_Bflat now
    return psi

In [483]:
psi = MPS_state([x1,x2,x3],1,1,3)
print(psi)
psi.sites[1].leg

MPS, L=3, bc='finite'.
chi: [1, 1]
sites: SpinHalfSite(None) SpinHalfSite(None) SpinHalfSite(None)
forms: (0.0, 1.0) (0.0, 1.0) (0.0, 1.0)


LegCharge(ChargeInfo([], []), qconj=+1,
array([0, 2], dtype=int64), array([], shape=(1, 0), dtype=int32))

In [477]:
# construct TFIM directly
# use BosonSite
import tenpy.linalg.np_conserved as npc
from tenpy.networks.site import BosonSite as bs
from tenpy.models.lattice import Chain
from tenpy.networks.mpo import MPO, MPOEnvironment

L = 3
# Nmax is the maximum boson at one site, for TFIM is 1; 
# use BosonSite and chain module to construct sites
#site = bs(Nmax = 1, conserve = 'parity')
site = bs(Nmax = 1, conserve = None)
print(site.leg)
sites = L * [site]
lat = Chain(L, site, bc_MPS = 'finite')

N_max = 1
#print(site.B)
#print(site.Bd)
#print(site.N)
print(site.Id)
bX = (site.B + site.Bd) / 2
#print(bX)
bY = (site.B - site.Bd) / (2 * 1j)
#print(bY)
bZ = site.N - site.Id * N_max / 2
#print(bZ)
Id = site.Id

# construct the matrix/array which represents H_mpo: Ws
# transverse field ising model
g = 1.
J = 1.
d = 2
W_grid = [[Id, bX, -g*bZ],
          [None, None, -J*bX],
          [None, None, Id]]
p_leg = site.leg
print('***')
print(p_leg)
chinfo = p_leg.chinfo
print('***')
print(chinfo)
mpo_leg = npc.LegCharge.from_qflat(chinfo, [[], [], []])
W = npc.grid_outer(W_grid, [mpo_leg, mpo_leg.conj()], grid_labels=['wL', 'wR'])
Ws = [W] * L
Ws[0] = W[:1, :]
Ws[-1] = W[:, -1:]
print(Ws)
print('***')
H = MPO(sites, Ws, 'finite', IdL = 0, IdR = -1)
#H = MPO(sites, Ws, 'finite')
print(H)

 +1
0 []
2   
<npc.Array shape=(2, 2) charge=ChargeInfo([], []) labels=['p', 'p*']
 +1 | -1 
0 []|0 []
2   |2   
[[1. 0.]
 [0. 1.]]
>
***
 +1
0 []
2   
***
ChargeInfo([], [])
[<npc.Array shape=(1, 3, 2, 2) charge=ChargeInfo([], []) labels=['wL', 'wR', 'p', 'p*']>, <npc.Array shape=(3, 3, 2, 2) charge=ChargeInfo([], []) labels=['wL', 'wR', 'p', 'p*']>, <npc.Array shape=(3, 1, 2, 2) charge=ChargeInfo([], []) labels=['wL', 'wR', 'p', 'p*']>]
***


In [478]:
from tenpy.networks.mpo import MPO, MPOEnvironment
env = MPOEnvironment(psi, H, psi)
E = env.full_contraction(L-1)
return E

ValueError: incompatible LegCharge
self | other
 -1  |  -1  
0 [] | 0 [] 
2    | 1    

In [485]:
from tenpy.networks.mps import MPS
from tenpy.networks.site import BosonSite, SpinHalfSite
from tenpy.models.lattice import Chain
import tenpy.linalg.np_conserved as npc

def MPS_state(UM_list, d_phys, d_bond, L):
    '''
    construct the Matrix Product state for the whole chain (1d translational invariant bosonic chain) (can be adjusted)
    
    Parameters
    ----------
    UM_list: list of numpy.ndarray matrices
    
    returns the MPS type in TenPy
    '''
    #site = BosonSite(Nmax = 1, conserve = None)
    site = SpinHalfSite(conserve = None)
    lat = Chain(L, site, bc_MPS = 'finite')
    state = Site_Ten(UM_list[0], 0, d_phys, d_bond, L)
    for i in range(1, L):
        state1 = Site_Ten(UM_list[i], i, d_phys, d_bond, L)
        state = npc.tensordot(state, state1, ('vR','vL'))
    psi = MPS.from_full(lat.mps_sites(), state, 'B')
    # other methods to consider, MPS.from_Bflat, MPS
    # problem with MPS.from_Bflat now
    return psi

In [486]:
Jval = 1.
gval = 1.
Lval = 3
L = 3
bc = 'finite'
# built-in module TFIM and built-in function calc_H_MPO()
#TFI1 = TFIModel({'L':L,'J':Jval,'g':gval,'bc_MPS':'finite','conserve':"parity"})
TFI1 = TFIModel({'L':Lval,'J':Jval,'g':gval,'bc_MPS':'finite','conserve':None})
H_mpo = TFI1.calc_H_MPO()
print(H_mpo)

env = MPOEnvironment(psi,H_mpo,psi)
E = env.full_contraction(L-1)
print("E=", E)

Reading 'bc_MPS'='finite' for config TFIModel
Reading 'conserve'=None for config TFIModel
Reading 'L'=3 for config TFIModel
Reading 'J'=1.0 for config TFIModel
Reading 'g'=1.0 for config TFIModel


ValueError: incompatible LegCharge
self | other
 -1  |  -1  
0 [] | 0 [] 
2    | 1    

In [391]:
M = TFIChain({"L": L_val, "J": J_val, "g": g_val, "bc_MPS": 'finite'})
print(M)
H_mpo = M.calc_H_MPO()
print(H_mpo)
print(H_mpo.sites)
#print(H_mpo.Ws)
print(H_mpo.bc)
print(H_mpo.IdL)
print(H_mpo.IdR)
#H_mpo.IdL = [0,0,0]
#H_mpo.IdR = [-1,-1,-1]
print(H_mpo.IdL)
print(H_mpo.IdR)
print(H_mpo.get_IdL(1))
print(H_mpo.get_IdR(1))
print(H_mpo.get_IdL(0))
print(H_mpo.get_IdR(0))
print(H_mpo.chinfo)
#
# test for MPS.from_full
from tenpy.networks.site import SpinHalfSite
from tenpy.models.lattice import Chain
import tenpy.linalg.np_conserved as npc
site = SpinHalfSite()
L = 2
lat = Chain(L, site, bc_MPS = 'finite')
state = npc.tensordot(x_t, x_t1, ('vR','vL'))
psi = MPS.from_full(lat.mps_sites(), state)

print(site)
print(lat)
print(type(state))
print(type(psi))
print(psi)

Reading 'bc_MPS'='finite' for config TFIChain
Reading 'L'=2 for config TFIChain
Reading 'J'=1 for config TFIChain
Reading 'g'=1 for config TFIChain
[SpinHalfSite('parity'), SpinHalfSite('parity')]
finite
[0, 0, None]
[None, 2, 0]
[0, 0, None]
[None, 2, 0]
0
0
0
2
ChargeInfo([2], ['parity'])
SpinHalfSite('Sz')
<class 'tenpy.linalg.np_conserved.Array'>
<class 'tenpy.networks.mps.MPS'>
MPS, L=2, bc='finite'.
chi: [2]
sites: SpinHalfSite('Sz') SpinHalfSite('Sz')
forms: (1.0, 0.0) (0.0, 1.0)


In [306]:
L = L_val
from tenpy.networks.mpo import MPO, MPOEnvironment
env = MPOEnvironment(psi, H_mpo, psi)
E = env.full_contraction(L-1)
return E
# <tenpy.networks.mpo.MPOEnvironment object at 0x0000027C2996C488>

ValueError: could not broadcast input array from shape (0,0) into shape (0)